Design idea:
- When instantiated with data that it should process, it should process and store the result in self
- When instantiated without data it just store the parameters
- When called with data, it should create a new instance which stores that data
    - If you do all processing in init (or subfunctions), call only gets parameters and creates a new object.

In [15]:
class Spectrogram:
    # def __new__(cls, data=None, param=None):

    def __init__(self, data=None, param=None):
        print(f"Calling init with {data=}, {param=}")
        self.param = param
        # if isinstance(data, type(self)):
        #     print("Storing data in init")
        #     self.data = data.data
        if data is not None:
            print("Processing in init")
            self.data = data + param
        print("init done")

    def __call__(self, data):
        print("Calling")
        return type(self)(data, self.param)

    def __repr__(self):
        try:
            return f"Spectrogram(data={self.data}, param={self.param})"
        except AttributeError:
            return f"Spectrogram(param={self.param})"

f = Spectrogram(param='param')
d = f("data")
f2 = Spectrogram(f)

display(f)
display(d)

Calling init with data=None, param='param'
init done
Calling
Calling init with data='data', param='param'
Processing in init
init done
Calling init with data=Spectrogram(param=param), param=None
Storing data in init


AttributeError: 'Spectrogram' object has no attribute 'data'

In [10]:
class C:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __repr__(self):
        return f"{type(self).__name__}({self.start}, {self.stop})"
    def __and__(self, other):
        if not isinstance(other, type(self)):
            return NotImplemented
        start = max(self.start, other.start)
        stop = min(self.stop, other.stop)
        if start >= stop:
            raise ValueError("No overlap")
        return type(self)(start, stop)

a = C(0, 5)
b = C(1, 4)
display(a & b)
display(b & a)
c = C(-1, 3)
display(a & c)
display(b & c)
all([a, b, c])
# a & 5

C(1, 4)

C(1, 4)

C(0, 3)

C(1, 3)

True

In [121]:
import numpy as np
import xarray as xr
class C(np.lib.mixins.NDArrayOperatorsMixin):
    # __array_priority__= 100

    def __init__(self, x):
        self.x = x
    def __repr__(self):
        return f"'C' object with \n{self.x}"
    def __array__(self, dtype=None):
        print(f"calling array, {dtype = }")
        return self.x.__array__(dtype=dtype)

    # def __array_finalize__(self, obj):
    #     print('In __array_finalize__:')
    #     print('   self is %s' % repr(self))
    #     print('   obj is %s' % repr(obj))
    #     if obj is None: return
    #     self.info = getattr(obj, 'info', None)

    def __array_wrap__(self, out_arr, context=None):
        print('In __array_wrap__:')
        print('   self is %s' % repr(self))
        print('   arr is %s' % repr(out_arr))
        return type(self)(out_arr)
        return type(self)(self.x.__array_wrap__(out_arr, context=context))

    # def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
    #     print(f"calling __array_ufunc__ with {ufunc = }, {method = }, {inputs = }, {kwargs = }")
    #     inputs = (arg.x if isinstance(arg, C) else arg for arg in inputs )
    #     return type(self)(self.x.__array_ufunc__(ufunc, method, *inputs, **kwargs))

    # def __array_function__(self, func, types, args, kwargs):
    #     print(f"calling __array_function__ with {func=}, {types=}, {args=}, {kwargs=}")
    #     if func is np.sum:
    #         return self.sum()
    #     args = (arg.x if isinstance(arg, C) else arg for arg in args)
    #     return func(*args)

    def mean(self, dim=None, axis=None, **kwargs):
        print(f"calling mean with {dim=}, {axis=}, {kwargs=}")
        # then just call the parent

    def sum(self, *args, **kwargs):
        print("calling sum")

    def take(self, *args, **kwargs):
        print("calling take")

    def sort(self, *args, **kwargs):
        print("calling sort")

x = C(xr.DataArray(np.arange(10).reshape((2, 5)) + 1))
display("plain add:")
# display(x + x.x)

display("np.add")
display(np.add(x, 5))

display("np.sum")
display(np.sum(x))

display("np.abs")
display(np.abs(x))

# display("10 * np.log10")
# display(10 * np.log10(x))

display("np.mean")
display(np.mean(x))

display("np.take")
display(np.take(x, [0, 1]))
# display("np.std")
# display(np.std(x, axis=0))
# print()
# np.mean(x, dim="time")
# np.mean(x, 0)

# def dB(x, power=True, safe_zeros=True, ref=1):
#     '''Calculate the decibel of an input value

#     Parameters
#     ----------
#     x : numeric
#         The value to take the decibel of
#     power : boolean, default True
#         Specifies if the input is a power-scale quantity or a root-power quantity.
#         For power-scale quantities, the output is 10 log(x), for root-power quantities the output is 20 log(x).
#         If there are negative values in a power-scale input, the handling can be controlled as follows:
#         - `power='imag'`: return imaginary values
#         - `power='nan'`: return nan where power < 0
#         - `power=True`: as `nan`, but raises a warning.
#     safe_zeros : boolean, default True
#         If this option is on, all zero values in the input will be replaced with the smallest non-zero value.
#     ref : numeric
#         The reference unit for the decibel. Note that this should be in the same unit as the `x` input,
#         e.g., if `x` is a power, the `ref` value might need squaring.
#     '''
#     # if isinstance(x, _DataWrapper):
#     #     new = dB(x.data, power=power, safe_zeros=safe_zeros, ref=ref)
#     #     new = type(x)(new)
#     #     x._transfer_attributes(new)
#     #     return new
#     if safe_zeros and np.size(x) > 1:
#         nonzero = x != 0
#         min_value = np.nanmin(abs(xr.where(nonzero, x, np.nan)))
#         x = xr.where(nonzero, x, min_value)
#         print("in safe seros:", x)
#     if power:
#         if np.any(x < 0):
#             if power == 'imag':
#                 return 10 * np.log10(x + 0j)
#             if power == 'nan':
#                 return 10 * np.log10(xr.where(x > 0, x, np.nan))
#         return 10 * np.log10(x / ref)
#     else:
#         return 20 * np.log10(np.abs(x) / ref)

# dB(x.x)

'plain add:'

'np.add'

calling array, dtype = None
In __array_wrap__:
   self is 'C' object with 
<xarray.DataArray (dim_0: 2, dim_1: 5)>
array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10]])
Dimensions without coordinates: dim_0, dim_1
   arr is array([[ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15]])


'C' object with 
[[ 6  7  8  9 10]
 [11 12 13 14 15]]

'np.sum'

...

calling mean with dim=None, axis=None, kwargs={'dtype': None, 'out': None}


None

'np.take'

calling take


None

In [111]:
x.x.__array_function__

AttributeError: 'DataArray' object has no attribute '__array_function__'

In [2]:
import xarray as xr
xr.DataArray.

Creating wrapper for func =  <function D.operate at 0x000001E0B6995D00>
calling wrapper with self = <__main__.D object at 0x000001E0B692CA10>, args = ('working great',), kwargs = {}
operating on value = 'working great' from self = 'not an instanse'
returning from wrapper


In [174]:
class C:
    def __init__(self, x):
        self.x = x

    def __array_function__(self, func, types, args, kwargs):
        if func is np.mean:
            return self.mean()
        return NotImplemented

    def mean(self):
        print(f"calling C.mean on {self}")
        return self.x.mean()

class D(C):
    def __array_function__(self, func, types, args, kwargs):
        if func is np.sum:
            return self.sum()
        return super().__array_function__(func, types, args, kwargs)
    def sum(self):
        print(f"calling D.sum on {self}")
        return type(self)(self.x.sum())

c = C(np.arange(5))
np.mean(c)
d = D(np.arange(5))
np.sum(d)
np.mean(d)

calling C.mean on <__main__.C object at 0x000001E0B6ED7EF0>
calling D.sum on <__main__.D object at 0x000001E0B3DEDAF0>
calling C.mean on <__main__.D object at 0x000001E0B3DEDAF0>


2.0

In [6]:
import xarray as xr
import uwacan
class Sensor(uwacan._core.DatasetWrap):
    def __new__(cls, sensor, /, position=None, sensitivity=None, depth=None, latitude=None, longitude=None):
        if isinstance(sensor, Sensor):
            sensor = sensor._data
        if isinstance(sensor, xr.Dataset):
            sensor = sensor[[key for key, value in sensor.notnull().items() if value]]
            if "latitude" in sensor and "longitude" in sensor:
                cls = SensorPosition
                # obj = _SensorPosition(sensor)
            else:
                cls = Sensor
                # obj = _Sensor(sensor)
        else:
            if position is not None or (latitude is not None and longitude is not None):
                cls = SensorPosition
                sensor = xr.Dataset(data_vars=dict(latitude=latitude, longitude=longitude), coords={"sensor": sensor})
                # obj = _SensorPosition(position, latitude=latitude, longitude=longitude)
            else:
                cls = Sensor
                sensor = xr.Dataset(coords={"sensor": sensor})
                # obj = _Sensor(xr.Dataset())
            # obj._data.coords["sensor"] = sensor

        if "sensor" not in sensor:
            raise ValueError("Cannot have unlabeled sensors")
        if sensitivity is not None:
            sensor["sensitivity"] = sensitivity
        if depth is not None:
            sensor["depth"] = depth
        # return cls(sensor)
        return super().__new__(cls)

    def __init__(self, data, **kwargs):
        print("initializing sensor")
        super().__init__(data)
        # self.data = data

class SensorPosition(Sensor, uwacan.positional.Position):
    def __init__(self, data, **kwargs):
        print("initializing sensor pos")
        super().__init__(data)

Sensor("S1", depth=54, latitude=58, longitude=11).data

initializing sensor pos
initializing sensor


ValueError: Cannot parse coordinate string 'S1'

In [18]:
ds = xr.Dataset(data_vars={"x": 5, "y": 7, "z": [1, 2]})
sel = ds.isel(z=0)
display(sel)
del sel["x"]
display(ds)
display(sel)

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    z        int32 1
Data variables:
    x        int32 5
    y        int32 7

<xarray.Dataset>
Dimensions:  (z: 2)
Coordinates:
  * z        (z) int32 1 2
Data variables:
    x        int32 5
    y        int32 7

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    z        int32 1
Data variables:
    y        int32 7

In [15]:
import xarray as xr
import uwacan
class Sensor(uwacan._core.DatasetWrap):
    def __new__(cls, sensor, /, position=None, sensitivity=None, depth=None, latitude=None, longitude=None):
        if isinstance(sensor, Sensor):
            sensor = sensor._data
        if isinstance(sensor, xr.Dataset):
            # for key, value in sensor.notnull().items():
            #     if not value:
            #         del sensor[key]

            sensor = sensor[[key for key, value in sensor.notnull().items() if value]]
            if "latitude" in sensor and "longitude" in sensor:
                cls = SensorPosition
                # obj = _SensorPosition(sensor)
            else:
                cls = Sensor
                # obj = _Sensor(sensor)
        else:
            if position is not None or (latitude is not None and longitude is not None):
                cls = SensorPosition
                sensor = xr.Dataset(data_vars=dict(latitude=latitude, longitude=longitude), coords={"sensor": sensor})
                # obj = _SensorPosition(position, latitude=latitude, longitude=longitude)
            else:
                cls = Sensor
                sensor = xr.Dataset(coords={"sensor": sensor})
                # obj = _Sensor(xr.Dataset())
            # obj._data.coords["sensor"] = sensor

        if "sensor" not in sensor:
            raise ValueError("Cannot have unlabeled sensors")
        if sensitivity is not None:
            sensor["sensitivity"] = sensitivity
        if depth is not None:
            sensor["depth"] = depth
        # return cls(sensor)
        return super().__new__(cls)

    def __init__(self, sensor, /, sensitivity=None, depth=None):
        print("initializing sensor")
        if isinstance(sensor, Sensor):
            sensor = sensor._data
        if isinstance(sensor, xr.Dataset):
            sensor = sensor[[key for key, value in sensor.notnull().items() if value]]
        else:
            sensor = xr.Dataset(coords={"sensor": sensor})
        super().__init__(sensor)

        if "sensor" not in self.data:
            raise ValueError("Cannot have unlabeled sensors")
        if sensitivity is not None:
            self.data["sensitivity"] = sensitivity
        if depth is not None:
            self.data["depth"] = depth
        # self.data = data

class SensorPosition(Sensor, uwacan.positional.Position):
    def __init__(self, sensor, /, position=None, sensitivity=None, depth=None, latitude=None, longitude=None):
        Sensor.__init__(self, sensor, sensitivity=sensitivity, depth=depth)
        latitude, longitude = self._parse_coordinates(position, latitude=latitude, longitude=longitude)
        self.data["latitude"] = latitude
        self.data["longitude"] = longitude
        # print("initializing sensor pos")
        # super().__init__(data)

s = Sensor("S1", depth=54, position="58°30'N 11°30'E")
display(s)
display(s.data)

initializing sensor


SensorPosition(58.5000, 11.5000)

<xarray.Dataset>
Dimensions:    ()
Coordinates:
    sensor     <U2 'S1'
Data variables:
    depth      int32 54
    latitude   float64 58.5
    longitude  float64 11.5

In [21]:
class C:
    """A docstring"""

    @property
    def x(self):
        """A docstring"""


In [183]:
class C:
    @staticmethod
    def implements(np_func):
        def decorator(func):
            func._implements = np_func
            return func
        return decorator

    def __init__(self, x):
        self.x = x

    def __init_subclass__(cls) -> None:
        print(f"calling init subclass {cls}")
        implementations = {}
        for name, value in cls.__dict__.items():
            if callable(value) and hasattr(value, "_implements"):
                implementations[value._implements] = value
                print(name, value)
        cls._implementations = implementations

    def __array_function__(self, func, types, args, kwargs):
        for cls in self.__class__.mro():
            if hasattr(cls, f"_implementations"):
                if func in cls._implementations:
                    func = cls._implementations[func]
                    break
        else:
            return NotImplemented
        return func(*args, **kwargs)

    @implements(np.mean)
    def mean(self):
        print("calling C.mean")
        return self.x.mean()

    def sum(self):
        print("calling C.sum")

C.__init_subclass__()

class D(C):
    @C.implements(np.sum)
    def sum(self):
        print("calling D.sum")
        return super().sum()
        return type(self)(self.x.sum())

c = C(np.arange(5))
np.mean(c)
d = D(np.arange(5))
np.sum(d)
np.mean(d)

calling init subclass <class '__main__.C'>
mean <function C.mean at 0x000001E0B701EDE0>
calling init subclass <class '__main__.D'>
sum <function D.sum at 0x000001E0B701D1C0>
calling C.mean
calling D.sum
calling C.sum
calling C.mean


2.0

In [182]:
dir(D)

['_C__implementations',
 '__array_function__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'implements',
 'mean',
 'sum']

In [127]:
class C:
    def __init_subclass__(cls) -> None:
        print(f"calling init subclass with {cls}")

class D(C):
    pass

calling init subclass with <class '__main__.D'>


In [122]:
xr.DataArray.__array_ufunc__??

Signature: xr.DataArray.__array_ufunc__(self, ufunc, method, *inputs, **kwargs)
Docstring: <no docstring>
Source:   
    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        from xarray.core.computation import apply_ufunc

        # See the docstring example for numpy.lib.mixins.NDArrayOperatorsMixin.
        out = kwargs.get("out", ())
        for x in inputs + out:
            if not is_duck_array(x) and not isinstance(
                x, self._HANDLED_TYPES + (SupportsArithmetic,)
            ):
                return NotImplemented

        if ufunc.signature is not None:
            raise NotImplementedError(
                f"{ufunc} not supported: xarray objects do not directly implement "
                "generalized ufuncs. Instead, use xarray.apply_ufunc or "
                "explicitly convert to xarray objects to NumPy arrays "
                "(e.g., with `.values`)."
            )

        if method != "__call__":
            # TODO: support other methods,

In [73]:
np.__version__

'1.26.3'

In [72]:
dir(np.lib.mixins)

['NDArrayOperatorsMixin',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_binary_method',
 '_disables_array_ufunc',
 '_inplace_binary_method',
 '_numeric_methods',
 '_reflected_binary_method',
 '_unary_method',
 'um']

In [53]:
np.testing.overrides.get_overridable_numpy_ufuncs()

{<ufunc '_ones_like'>,
 <ufunc 'absolute'>,
 <ufunc 'add'>,
 <ufunc 'arccos'>,
 <ufunc 'arccosh'>,
 <ufunc 'arcsin'>,
 <ufunc 'arcsinh'>,
 <ufunc 'arctan'>,
 <ufunc 'arctan2'>,
 <ufunc 'arctanh'>,
 <ufunc 'bitwise_and'>,
 <ufunc 'bitwise_or'>,
 <ufunc 'bitwise_xor'>,
 <ufunc 'cbrt'>,
 <ufunc 'ceil'>,
 <ufunc 'clip'>,
 <ufunc 'conjugate'>,
 <ufunc 'copysign'>,
 <ufunc 'cos'>,
 <ufunc 'cosh'>,
 <ufunc 'deg2rad'>,
 <ufunc 'degrees'>,
 <ufunc 'divide'>,
 <ufunc 'divmod'>,
 <ufunc 'equal'>,
 <ufunc 'exp'>,
 <ufunc 'exp2'>,
 <ufunc 'expm1'>,
 <ufunc 'fabs'>,
 <ufunc 'float_power'>,
 <ufunc 'floor'>,
 <ufunc 'floor_divide'>,
 <ufunc 'fmax'>,
 <ufunc 'fmin'>,
 <ufunc 'fmod'>,
 <ufunc 'frexp'>,
 <ufunc 'gcd'>,
 <ufunc 'greater'>,
 <ufunc 'greater_equal'>,
 <ufunc 'heaviside'>,
 <ufunc 'hypot'>,
 <ufunc 'invert'>,
 <ufunc 'isfinite'>,
 <ufunc 'isinf'>,
 <ufunc 'isnan'>,
 <ufunc 'isnat'>,
 <ufunc 'lcm'>,
 <ufunc 'ldexp'>,
 <ufunc 'left_shift'>,
 <ufunc 'less'>,
 <ufunc 'less_equal'>,
 <ufunc 'log

In [36]:
xr.DataArray.__array_priority__?

Type:        int
String form: 60
Docstring:  
int([x]) -> integer
int(x, base=10) -> integer

Convert a number or string to an integer, or return 0 if no arguments
are given.  If x is a number, return x.__int__().  For floating point
numbers, this truncates towards zero.

If x is not a number or if base is given, then x must be a string,
bytes, or bytearray instance representing an integer literal in the
given base.  The literal can be preceded by '+' or '-' and be surrounded
by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
Base 0 means to interpret the base from the string as an integer literal.
>>> int('0b100', base=0)
4